<a href="https://colab.research.google.com/github/Beccacec/FDS-Final-Project/blob/Lorenzo/FDS_final_project_Lorenzo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import csv
import pandas as pd

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


We select specific features in order to find out which of them can affect the price of the property.
So we choose features like "id", "name", "neighbourhood" (because some of them are not in specifically in the city of Rome"), "price" and others that seemed interesting for this analysis.

In [29]:
features_price = ["id","name","neighbourhood","neighbourhood_cleansed","latitude","longitude","property_type","room_type","bedrooms","beds","amenities","price","number_of_reviews_ltm","review_scores_rating"]

In [30]:
df = pd.read_csv("/content/drive/MyDrive/FDS_final_project/listings.csv", usecols = features_price)
df.head()

,id,name,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,bedrooms,beds,amenities,price,number_of_reviews_ltm,review_scores_rating
0,17629440,ISOLA DEL CUORE,"Ciampino, Lazio, Italy",VII San Giovanni/Cinecittà,41.78984,12.60899,Private room in rental unit,Private room,1.0,1.0,"[""Heating"", ""Shampoo"", ""Cleaning products"", ""H...",$45.00,8,4.74
1,2737,"Elif's room in cozy, clean flat.","Rome, Lazio, Italy",VIII Appia Antica,41.87136,12.48215,Private room,Private room,1.0,1.0,"[""First aid kit"", ""Heating"", ""Elevator"", ""Wifi...",$50.00,0,4.80
2,103870,CASACHICA Your holiday home in Rome,NaN,I Centro Storico,41.89711,12.50423,Entire rental unit,Entire home/apt,1.0,2.0,"[""Air conditioning"", ""Heating"", ""Lockbox"", ""Mi...",$78.00,30,4.11
3,2903,Stay a night in Calisto6 B&B Rome,NaN,I Centro Storico,41.88824,12.46953,Private room in bed and breakfast,Private room,1.0,2.0,"[""Air conditioning"", ""First aid kit"", ""Heating...",$164.00,0,4.28
4,104339,Colorful apartment close MetroB,"Rome, Lazio, Italy",VIII Appia Antica,41.85585,12.48361,Entire rental unit,Entire home/apt,1.0,2.0,"[""Heating"", ""Paid parking off premises"", ""Lugg...",$80.00,0,5.00
